# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging

from matplotlib import pyplot as plt
import pandas as pd
import os

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.automl.core.featurization import FeaturizationConfig
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig
from azureml.interpret import ExplanationClient
from azureml.automl.runtime.onnx_convert import OnnxConverter
from azureml.data.dataset_factory import TabularDatasetFactory
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from azureml.core.run import Run
from sklearn.model_selection import train_test_split
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn

print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
ws.write_config(path='.azureml')

# choose a name for experiment
experiment_name = 'automl-classification-experiment'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-135159
Azure region: southcentralus
Subscription id: a24a24d5-8d87-4c8a-99b6-91ed2d2df51f
Resource group: aml-quickstarts-135159


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = "cpu-cluster-1"
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True)

Found existing cluster


In [4]:
key = 'uciheartfailuredataset'
if key in ws.datasets.keys(): 
    dataset = ws.datasets[key] 
    print("Found dataset")
else:
    url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv'
    dataset = Dataset.Tabular.from_delimited_files(url)
    dataset = dataset.register(workspace=ws, name=key)

Found dataset


In [5]:
ds = dataset.to_pandas_dataframe()
train, test = train_test_split(ds, random_state=625, shuffle=True)

In [6]:
train.to_csv('train.csv', index=False)
datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./train.csv'])
train = Dataset.Tabular.from_delimited_files(path = [(datastore,'train.csv')])

Uploading an estimated of 1 files
Uploading ./train.csv
Uploaded ./train.csv, 1 files out of an estimated total of 1
Uploaded 1 files


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_hours" : 0.3,
    "enable_early_stopping" : True,
    "iteration_timeout_minutes": 5,
    "max_concurrent_iterations": 4,
    "max_cores_per_iteration": -1,
    "primary_metric": 'accuracy',
    "featurization": 'auto',
    "verbosity": logging.INFO,
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    debug_log = 'automl_errors.log',
    compute_target=compute_target,
    task="classification",
    training_data= train,
    label_column_name="DEATH_EVENT",
    enable_onnx_compatible_models=True,
    n_cross_validations= 3,
    **automl_settings)

In [8]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config,show_output = False)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [10]:
%%time
remote_run.wait_for_completion()

CPU times: user 32.8 ms, sys: 7.17 ms, total: 40 ms
Wall time: 3.16 s


{'runId': 'AutoML_a82676aa-e08f-4e93-8432-f74183ff9f8f',
 'target': 'cpu-cluster-1',
 'status': 'Completed',
 'startTimeUtc': '2021-01-18T06:22:29.607108Z',
 'endTimeUtc': '2021-01-18T06:45:48.529515Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'cpu-cluster-1',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-classification-experiment","subscription_id":"a24a24d5-8d87-4c8a-99b6-91ed2d2df51f","resource_group":"aml-quickstarts-135159","workspace_name":"quick-starts-ws-135159","region":"southcentralus","compute_target":"cpu-cluster-1","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":null,"y_max":n

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.

## Retrieve the Best ONNX Model


In [11]:
best_run, onnx_model= remote_run.get_output(return_onnx_model=True)

In [12]:
#TODO: Save the best model
OnnxConverter.save_onnx_model(onnx_model, file_path="./automl_model.onnx")


In [13]:
best_run.get_properties()

{'runTemplate': 'automl_child',
 'pipeline_id': '__AutoML_Ensemble__',
 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'automl-classification-experiment\',\'compute_target\':\'cpu-cluster-1\',\'subscription_id\':\'a24a24d5-8d87-4c8a-99b6-91ed2d2df51f\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_a82676aa-e08f-4e93-8432-f74183ff9f8f_35","experiment_name":"automl-classification-experiment","workspace_name":"quick-starts-ws-135159","subscription_id":"a24a24d5-8d87-4c8a-99b6-91ed2d2df51f","resource_group_name":"aml-quickstarts-135159"}}]}',
 'training_percent': '100',
 'predicted_cost': None,
 'iteration': '35',
 '_aml_system_scenario_identification': 'Rem

In [14]:
best_run.get_metrics()

{'precision_score_weighted': 0.8966007939706552,
 'precision_score_micro': 0.8926726726726727,
 'AUC_macro': 0.9208363636363636,
 'f1_score_weighted': 0.8898748237676449,
 'f1_score_macro': 0.8681244681497461,
 'AUC_weighted': 0.9208363636363636,
 'recall_score_micro': 0.8926726726726727,
 'recall_score_macro': 0.8511285091719873,
 'accuracy': 0.8926726726726727,
 'average_precision_score_micro': 0.9334823072770367,
 'log_loss': 0.33775671659932316,
 'f1_score_micro': 0.8926726726726727,
 'average_precision_score_weighted': 0.9326774589323062,
 'AUC_micro': 0.933597413629846,
 'weighted_accuracy': 0.9237466491714276,
 'precision_score_macro': 0.8983069177877834,
 'matthews_correlation': 0.7472628995235754,
 'average_precision_score_macro': 0.9161482771638978,
 'norm_macro_recall': 0.702257018343975,
 'balanced_accuracy': 0.8511285091719873,
 'recall_score_weighted': 0.8926726726726727,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_a82676aa-e08f-4e93-8432-f74183ff9f8f_3

In [15]:
print(best_run)

Run(Experiment: automl-classification-experiment,
Id: AutoML_a82676aa-e08f-4e93-8432-f74183ff9f8f_35,
Type: azureml.scriptrun,
Status: Completed)


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [16]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')

In [17]:
description = 'AutoML Model'
tags = None
model = remote_run.register_model(model_name = model_name, description = description, tags = tags)
print(remote_run.model_id) 

AutoMLa82676aae35


In [18]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

inference_config = InferenceConfig(entry_script='inference/score.py')

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {'type': "automl_classification"}, 
                                               description = 'Heart Data Service')

aci_service_name = 'automl-uciheart1'
print(aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

automl-uciheart1
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..........................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


## Enable Application Insitghts on The Deployed Service

In [19]:
aci_service.update(enable_app_insights=True)

## Service Details

In [20]:
print(f'\nservice state: {aci_service.state}\n')
print(f'scoring URI: \n{aci_service.scoring_uri}\n')
print(f'swagger URI: \n{aci_service.swagger_uri}\n')


service state: Healthy

scoring URI: 
http://3939b333-1264-4b3e-ac04-029c9d6003af.southcentralus.azurecontainer.io/score

swagger URI: 
http://3939b333-1264-4b3e-ac04-029c9d6003af.southcentralus.azurecontainer.io/swagger.json



## Test Service : 
TODO: In the cell below, send a request to the web service you deployed to test it.

In [21]:
import requests
import json

scoring_uri = aci_service.scoring_uri
# Two sets of data to score, so we get two results back
data = {"data":
        [
          { 
            "age": 45,
            "anaemia": 0,
            "creatinine_phosphokinase": 2413,
            "diabetes": 0,
            "ejection_fraction": 38,
            "high_blood_pressure": 0,
            "platelets": 140000,
            "serum_creatinine": 1.4,
            "serum_sodium": 140,
            "sex": 1,
            "smoking": 1,
            "time": 280
          },
          {
            "age": 50,
            "anaemia": 0,
            "creatinine_phosphokinase": 196,
            "diabetes": 0,
            "ejection_fraction": "45",
            "high_blood_pressure": 0,
            "platelets": 395000,
            "serum_creatinine": 1.6,
            "serum_sodium": 136,
            "sex":1,
            "smoking": 1,
            "time": 285
          },
      ]
    }
# Convert to JSON string
input_data = json.dumps(data)
# Set the content type
headers = {'Content-Type': 'application/json'}
# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())

{"result": [0, 0]}


TODO: In the cell below, print the logs of the web service and delete the service

In [22]:
aci_service.get_logs()

'2021-01-18T06:55:41,753091119+00:00 - rsyslog/run \n2021-01-18T06:55:41,756330873+00:00 - iot-server/run \n2021-01-18T06:55:41,758454908+00:00 - nginx/run \n/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n2021-01-18T06:55:41,762246071+00:00 - gunicorn/run 

In [25]:
aci_service.delete()
model.delete()
compute_target.delete()

No service with name automl-uciheart1 found to delete.
Current provisioning state of AmlCompute is "Deleting"

